# Train a Simplifed AlexNet to classify CIFAR-10 data¶
## Import python libs and setup FLAGS

In [1]:
from cifar10 import cifar10
import os
import tensorflow as tf
import tf_general as tfg
import numpy as np
from train_log import train_log

FLAGS = tf.flags.FLAGS
try:
    tf.app.flags.DEFINE_string('f', '', 'kernel')
    tf.flags.DEFINE_integer('epoch', 50000, 'epoch')
    tf.flags.DEFINE_integer('batch_size', 128, 'batch size')
    tf.flags.DEFINE_float('lr', 0.01, 'learning rate')
    tf.flags.DEFINE_float('drop_rate', 0.3, 'drop out rate for drop lay')
    tf.flags.DEFINE_boolean('augument', True, 'if image augument is applied') 
    tf.flags.DEFINE_integer('image_size', 225, 'image size')
    tf.flags.DEFINE_integer('test_size', 512, 'test size')
    #tf.flags.DEFINE_boolean('restore', False, 'restore from checkpoint and run test')
    print('parameters are defined.')
except:
    print('parameters have been defined.')
    
CONTINUE = 0
RUN = 1

parameters are defined.


## Section 2. Generate Checkpoint dir and Log dir

- Checkpoint dir is saved in variable **../Le-Net5-Log/Le-Net5_CLASS/ckpt_RUN**, if the dir doesn't exist then create it. 
- Log file dir is saved in variable **../Le-Net5-Log/Le-Net5_CLASS/log_RUN**, if the dir doesn't exist then create it. 
- data_path is the position of the CIFAR-10 image data
The reason to save the model and log outside the project is to avoid effect the git code management.

In [2]:
ckpt_dir = '../AlexNet-Log/AlexNet_CLASS/ckpt_'+str(RUN)+'/'
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

log_dir = '../AlexNet-Log/AlexNet_CLASS/log_'+str(RUN)+'/'
log = train_log(log_dir)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

data_path = '../cifar-10-batches-py/'
if not os.path.exists(data_path):
    print('The data path doesn\'t exist. Please check if it is a correct data path.')

In [3]:
class alexnet(object):

    def __init__(self, x, n_class=10, rate=0):
        self.input = x
        self.n_class = n_class
        self.drop_rate = rate
        self._build_net()

    def _build_net(self):
        with tf.name_scope('norm_1'):    
            self.x_norm1 = tfg.local_response_norm(self.input, depth_radius=4, bias=1.0, alpha=0.001/9, beta=0.75)
            print('norm_1: ', self.x_norm1.get_shape())
        
        with tf.name_scope('conv_1'):
            self.conv1 = tfg.conv2d(self.input, 11, 4, 96, 'conv_1', 'VALID','RELU')
            print('conv_1: ', self.conv1.get_shape())
        
        with tf.name_scope('lrn_1'):
            self.lrn1 = tfg.local_response_norm(self.conv1,depth_radius=4, 
                                                bias=1.0, alpha=0.001/9, beta=0.75)
            print('lrn_1: ', self.lrn1.get_shape())
            
        with tf.name_scope('pool_1'):
            self.pool1 = tfg.max_pool(self.lrn1, 3, 2, 'pool1', 'VALID')
            print('pool_1: ', self.pool1.get_shape())
            
        with tf.name_scope('conv_2'):
            self.conv2 = tfg.conv2d(self.pool1, 5, 1, 256, 'conv_2', 'SAME','RELU')
            print('conv_2: ', self.conv2.get_shape())
        
        with tf.name_scope('lrn_2'):
            self.lrn2 = tfg.local_response_norm(self.conv2,depth_radius=4, 
                                                bias=1.0, alpha=0.001/9, beta=0.75)
            print('lrn_2: ', self.lrn2.get_shape())
            
        with tf.name_scope('pool_2'):
            self.pool2 = tfg.avg_pool(self.lrn2, 3, 2, 'pool2', 'VALID')
            print('pool_2: ', self.pool2.get_shape())
            
        with tf.name_scope('conv_3'):
            self.conv3 = tfg.conv2d(self.pool2, 3, 1, 384, 'conv_3', 'SAME','RELU')
            print('conv_3: ', self.conv3.get_shape())

        with tf.name_scope('conv_4'):
            self.conv4 = tfg.conv2d(self.conv3, 3, 1, 256, 'conv_4', 'SAME', 'RELU')
            print('conv_4: ', self.conv4.get_shape())
        
        with tf.name_scope('conv_5'):
            self.conv5 = tfg.conv2d(self.conv4, 3, 1, 256, 'conv_5', 'SAME','RELU')
            print('conv_5: ', self.conv5.get_shape())
            
        with tf.name_scope('pool_3'):
            self.pool3 = tfg.max_pool(self.conv5, 3, 2, 'pool3', 'VALID')
            print('pool_3: ', self.pool3.get_shape())
            
        with tf.name_scope('flat_1'):
            self.flat1, self.flat_dim = tfg.flatten(self.pool3)
            print('flat_1: ', self.flat1.get_shape())         

        with tf.name_scope('fc_1'):
            self.fc1 = tfg.fc_layer(self.flat1, self.flat_dim, 4096, 'fc_1','RELU')
            print('fc_1: ', self.fc1.get_shape())

        with tf.name_scope('fc_2'):
            self.fc2 = tfg.fc_layer(self.fc1, 4096, 4096, 'fc_2','RELU')
            print('fc_2: ', self.fc2.get_shape())
        
        with tf.name_scope('fc_3'):
            self.fc3 = tfg.fc_layer(self.fc2, 4096, 10, 'fc_3','RELU')
            print('fc_3: ', self.fc3.get_shape())
            
        with tf.name_scope('drop_out_1'):
            self.drop1 = tfg.drop_out(self.fc3, self.drop_rate, 'drop_out_1')
            self.logits = self.drop1
            print('drop_out_1: ', self.drop1.get_shape())

        with tf.name_scope('prediction'):
            self.prediction = tf.nn.softmax(self.drop1)
            print('prediction: ', self.prediction.get_shape())

## define input layer

In [4]:
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 225,225,3], name='input_image')
    drop_rate = tf.placeholder(tf.float32, name='drop_rate')
    global_step = tf.placeholder(tf.int32, name='global_step')
    y_ = tf.placeholder(tf.int64, [None], name='labels')

## define alexnet object 

In [5]:
with tf.name_scope('prediction'):
    alex_net = alexnet(x, drop_rate)
    y = alex_net.prediction

norm_1:  (?, 225, 225, 3)
conv_1:  (?, 54, 54, 96)
lrn_1:  (?, 54, 54, 96)
pool_1:  (?, 26, 26, 96)
conv_2:  (?, 26, 26, 256)
lrn_2:  (?, 26, 26, 256)
pool_2:  (?, 12, 12, 256)
conv_3:  (?, 12, 12, 384)
conv_4:  (?, 12, 12, 256)
conv_5:  (?, 12, 12, 256)
pool_3:  (?, 5, 5, 256)
flat_1:  (?, 6400)
fc_1:  (?, 4096)
fc_2:  (?, 4096)
fc_3:  (?, 10)
drop_out_1:  (?, 10)
prediction:  (?, 10)


## Use cross entropy as loss

In [6]:
with tf.name_scope('cross_entropy'):
        cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,
                                                                              labels=y_, 
                                                                              name="cross_entropy_per_example"))

## Use optimizer to optimize parameters

In [7]:
with tf.name_scope('train_step'):
    train_step = tf.train.AdagradOptimizer(FLAGS.lr).minimize(cross_entropy)

## Calculate training accuracy

In [8]:
with tf.name_scope('train_accuracy'):
    prediction =tf.argmax(y, 1)
    accuracy = tf.reduce_mean(tf.cast( tf.equal(prediction,y_), tf.float32))

## Start session to run the training

In [9]:
import time

data = cifar10(data_path)
saver = tf.train.Saver(max_to_keep = 1)

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver(max_to_keep=1)
    if CONTINUE != 0:
        model_file=tf.train.latest_checkpoint(ckpt_dir)
        saver.restore(sess,model_file)
    for i in range(CONTINUE, FLAGS.epoch):
        train_image, train_label,_ = data.get_train_batch(FLAGS.batch_size,FLAGS.augument, 
                                                          new_size=(FLAGS.image_size,FLAGS.image_size), resize = True )
        i += 1 
        loss, _,accuracy_rate = sess.run([cross_entropy, train_step, accuracy], 
                                         feed_dict={drop_rate: FLAGS.drop_rate, x:train_image, y_:train_label})
        print('.',end='')    
        log.add_log('train_accuracy',i, accuracy_rate)
        log.add_log('train_loss',i, loss)
        if (i+1)%250 == 0 and i > 0:
            saver.save(sess,ckpt_dir+'cifar10_'+str(i+1)+'.ckpt',global_step=i+1)
            acc_accuracy = 0
            for j in range(int(10000/FLAGS.test_size)):                    
                test_image, test_label,test_index = data.get_test_batch(FLAGS.test_size,
                                                                       new_size=(FLAGS.image_size,FLAGS.image_size), resize = True)
                accuracy_rate, output = sess.run([accuracy,prediction],
                                                 feed_dict={drop_rate: 0, x:test_image, y_:test_label})
                acc_accuracy += accuracy_rate
                log.add_log('test_index',i, test_index)
                log.add_log('output',i, output)
            accuracy_rate = acc_accuracy/10000*FLAGS.test_size
            print()
            print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + 
                      ' iter ' + str(i) + ', Test accuracy:' +str(round(accuracy_rate*100,2))+'%')
            log.add_log('test_accuracy',i, accuracy_rate)
            log.SaveToFile()
    tf.reset_default_graph()

train set length: 50000
test set length: 10000
.........................................................................................................................................................................................................................................................
2019-06-19 17:20:44 iter 249, Test accuracy:9.71%
.

AssertionError: Do not use tf.reset_default_graph() to clear nested graphs. If you need a cleared graph, exit the nesting and create a new graph.